In [1]:
# 구글 코랩 환경에서 실행, 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 구글 드라이브 경로 지정
path = '/content/drive/My Drive/Colab Notebooks/2020-PoscoICT/Data/'

In [3]:
# 필요한 패키지 설치
!python3 -m pip install --upgrade pip

# 한자 변환
!pip install hanja

# # 형태소 기반 토크나이징 (Konlpy)
# !python3 -m pip install konlpy
# # mecab (ubuntu, mac 기준)
# # 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 한글 여부 판단
!pip install soynlp

!pip install kss

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 119 kB 1.8 MB/s 
     |████████████████████████████████| 265 kB 7.0 MB/s 
     |████████████████████████████████| 229 kB 8.9 MB/s 
  Created wheel for hanja: filename=hanja-0.13.2-py3-none-any.whl size=128337 sha256=3cec40f1beeeafefde2e5888261046f5f3f185d972d065eabb8a1f432d67dc66
  Stored in directory: /root/.cache/pip/wheels/75/db/97/80e6c3401f000ec7fa16c3169d5e8817c8fe8474fc1f83fe0c
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44103 sha256=bcf36ac020f5e57e54eb7044ae2840ac6cb1bbfcd29cec1e346940cf5068ae69
  Stored in directory: /root/.cache/pip/wheels/d8/9b/e7/75af463b873c119dd444151fc54a8e190c87993593e1fa194a
Successfully built hanja pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13

# 데이터 로드 및 전처리


## 데이터 로드

In [5]:
from typing import List

import pandas as pd
import itertools
import re, hanja
import kss
 
def load_sentences(df: pd.DataFrame = None) -> List[str]:
    """
    개별 댓글에서 </p>와 <p>로 구분되어 있는 여러 문장을 문장으로 쪼갬
    :param df: 전처리 대상 데이터 프레임
    :return: 문장이 분리된 데이터 프레임
    """
    all_sentences = []
    for news in df['News'].values:
        try:
          all_sentences.append(kss.split_sentences(news))
        except Exception as e:
          print(e)

    merged = list(itertools.chain(*all_sentences))

    return merged

In [6]:
train_df = pd.read_csv(path+'news_train.csv', sep='\t',encoding='utf-8')
test_df = pd.read_csv(path+'news_test.csv', sep='\t', encoding='utf-8')

print(train_df.shape)
print(test_df.shape)

(5000, 6)
(500, 6)


In [7]:
train_sentences = load_sentences(train_df)
test_sentences = load_sentences(test_df)

print(len(train_sentences), len(test_sentences))

81135 8186


In [ ]:
import itertools
results = [x.split('</p> <p>') for x in train_sentences]
merged = list(itertools.chain(*results))
train_sentences = [x.replace('</p>','').replace('<p>', '').strip() for x in merged]
train_sentences[:10]

In [26]:
results = [x.split('</p> <p>') for x in test_sentences]
merged = list(itertools.chain(*results))
test_sentences = [x.replace('</p>','').replace('<p>', '').strip() for x in merged]
test_sentences[:10]

['[월드 IT쇼 2009] 다채로운 부대행사‥해외 바이어와 1대 1 수출 상담',
 "'월드IT쇼(WIS)2009' 부대행사도 다채롭게 열린다.",
 '국내 정보기술(IT) 기업들의 해외시장 개척을 지원해주는 수출상담회는 물론 미국 조달시장에 대한 한국 IT기업의 진출을 돕기 위한 행사도 따로 마련된다.',
 '대학에서 개발한 신기술을 산 · 학 협력으로 연결해주는 자리도 있다.',
 '',
 'IT분야 중소기업의 세계시장 판로 개척을 지원하는 다양한 수출상담회가 풍성하게 열린다.',
 '특히 기술 및 제품 구매를 직접 관장하는 해외 바이어와 국내 기업 간 일대일 비즈니스 상담 방식으로 이뤄져 실질적인 성과가 클 것으로 기대를 모으고 있다.',
 '',
 "18일 오전 10시부터 오후 5시까지 코엑스에서 '해외 바이어 초청 수출상담회'가 열린다.",
 '월드IT쇼에 참가하는 해외 바이어들을 국내 기업과 연결해주는 행사다.']

## 데이터 전처리

In [8]:
import re

In [9]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

EMAIL_PATTERN = re.compile(r'''(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))''', re.VERBOSE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.VERBOSE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [10]:
def cleansing_other(sentence: str = None) -> str:
    """
    문장을 전처리 (이메일, URL, 공백 등 제거) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub(EMAIL_PATTERN, ' ', sentence)
    sentence = re.sub(URL_PATTERN, ' ', sentence)
    sentence = re.sub(MULTIPLE_SPACES, ' ', sentence)
    sentence = sentence.replace(", )", "")
    
    return sentence

def cleansing_chinese(sentence: str = None) -> str:
    """
    한자를 변환하는 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    # chinese character를 앞뒤로 괄호가 감싸고 있을 경우, 대부분 한글 번역임
    sentence = re.sub("\([\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]+\)", "", sentence)
    # 다른 한자가 있다면 한글로 치환
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sentence) is not None:
        sentence = hanja.translate(sentence, 'substitution')

    return sentence

def cleansing_special(sentence: str = None) -> str:
    """
    특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence: str = None) -> str:
    """
    숫자를 전처리(delexicalization) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [11]:
def preprocess_sent(sentence: str = None) -> str:
    """
    모든 전처리를 수행 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sent_clean = sentence
    sent_clean = cleansing_other(sent_clean)
    sent_clean = cleansing_chinese(sent_clean)
    sent_clean = cleansing_special(sent_clean)
    sent_clean = cleansing_numbers(sent_clean)
    sent_clean = re.sub('\s+', ' ', sent_clean)

    return sent_clean

In [ ]:
# 전처리 예시
new_sents = []
original_sents = test_sentences[-10:]
for sent in original_sents:
    new_sent = preprocess_sent(sent)
    new_sents.append(new_sent)

for ori, new in zip(original_sents, new_sents):
    print("----------")
    print(ori)
    print(new)

In [24]:
from typing import List
from tqdm import tqdm

def write_to_txt(sentences: List[str] = None,
                 filename: str = None):
    """
    전처리 완료된 데이터를 txt파일로 저장하는 함수
    :param sentences: 전처리 완료된 문장 리스트
    :param filename: 저장 파일 경로
    """
    with open(filename, 'w', encoding='utf-8') as f:
        for sent in tqdm(sentences):
            if sent != '':
              f.write(preprocess_sent(sent)+'\n')
    f.close()
    print(f'Data saved at {filename}')

In [28]:
write_to_txt(test_sentences, path + 'news_sentence_test.txt')

100%|██████████| 12825/12825 [00:00<00:00, 24562.51it/s]

Data saved at /content/drive/My Drive/Colab Notebooks/2020-PoscoICT/Data/news_sentence_test.txt
